# 1. Introdução

Este notebook tem como objetivo explorar e analisar bases de dados fornecidas para o Desafio Datathon da FIAP. O foco principal é compreender os dados disponíveis, realizar uma análise exploratória e propor um modelo de predição adequado ao desafio. Ao longo do notebook, serão documentados os principais insights, etapas de pré-processamento, seleção de variáveis e justificativas para as escolhas metodológicas, visando alcançar os melhores resultados preditivos possíveis.

## 1.2 Importações

In [24]:
#importações
import pandas as pd
import json
import os

# 2. Exploração das bases de dados

## 2.1 base applicantes.json
Applicants.json -> É chaveado pelo código do candidato e possui todas as informações referentes ao candidato: Informações básicas, pessoais, profissionais, formação e o cv. Informações importantes desse json:

Nível acadêmico, de inglês e espanhol
Conhecimentos técnicos
Área de atuação
Cv completo
 

Utilização: Por exemplo, a vaga 10976 (chave no Jobs.json), possui 25 prospecções (chave 10976 no prospects.json), onde o candidato “Sr. Thales Freitas”  (chave 41496 no applicants.json) foi contratado.

 

In [25]:
def load_json_to_df_applicants(file_path): # Nome da função ajustado para applicants
    """Carrega dados de candidatos (applicants) de um JSON, usando a chave numérica como índice."""
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"O arquivo '{file_path}' não foi encontrado.")
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        data_list = []
        for key, value in json_data.items():
            value['id_candidato'] = key # ID do candidato
            data_list.append(value)

        df = pd.json_normalize(data_list, sep='_')
        df.set_index('id_candidato', inplace=True)

        # Ajuste de colunas duplicadas do exemplo anterior, se aplicável
        # Essas colunas vieram das seções 'infos_basicas' e 'informacoes_pessoais'
        prefer_cols = {
            'nome': 'informacoes_pessoais_nome',
            'email': 'informacoes_pessoais_email',
            'telefone_recado': 'informacoes_pessoais_telefone_recado',
            'telefone_celular': 'informacoes_pessoais_telefone_celular'
        }
        for final_col, preferred_source_col in prefer_cols.items():
            if preferred_source_col in df.columns:
                df[final_col] = df[preferred_source_col]
        cols_to_drop = [
            'infos_basicas_nome', 'infos_basicas_email',
            'infos_basicas_telefone_recado', 'infos_basicas_telefone'
        ]
        df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

        return df
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Erro ao carregar candidatos (applicants): {e}")
        return pd.DataFrame() # Retorna um DataFrame vazio em caso de erro



In [26]:
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42482 entries, 31000 to 5999
Data columns (total 57 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   cv_pt                                             42482 non-null  object
 1   cv_en                                             42482 non-null  object
 2   infos_basicas_objetivo_profissional               42482 non-null  object
 3   infos_basicas_data_criacao                        42482 non-null  object
 4   infos_basicas_inserido_por                        42482 non-null  object
 5   infos_basicas_local                               42482 non-null  object
 6   infos_basicas_sabendo_de_nos_por                  42482 non-null  object
 7   infos_basicas_data_atualizacao                    42482 non-null  object
 8   infos_basicas_codigo_profissional                 42482 non-null  object
 9   informacoes_pessoais_data_acei

## 2.2 base vagas.json

vagas.json -> É chaveado pelo código da vaga e possui as informações referentes a vaga aberta no nosso ats divididas em Informações básicas, perfil da vaga e benefícios. Aqui temos dados importantes como, por exemplo:

indicação se é vaga SAP ou não
Cliente solicitante
Nível profissional e nível de idiomas requeridos
Principais atividades e competências técnicas requeridas



In [27]:
def load_json_to_df_vagas(file_path):
    """Carrega dados de vagas de um JSON, usando a chave numérica como índice."""
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"O arquivo '{file_path}' não foi encontrado.")
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        data_list = []
        for vaga_id, vaga_info in json_data.items():
            vaga_info['id_vaga'] = vaga_id
            data_list.append(vaga_info)

        df = pd.json_normalize(data_list, sep='_')
        df.set_index('id_vaga', inplace=True)
        return df
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Erro ao carregar vagas: {e}")
        return pd.DataFrame()


## 2.3 base prospects.json

Prospects.json -> Também é chaveado pelo código da vaga e possui todas as prospecções da vaga.

Lista de prospecções com o código, nome, comentário e situação do candidato na vaga em questão


In [28]:
def load_json_to_df_prospects(file_path):
    """Carrega dados de prospects de um JSON, achata a lista aninhada e usa o id_titulo como índice."""
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"O arquivo '{file_path}' não foi encontrado.")
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        temp_data_list = []
        for title_id, title_info in json_data.items():
            title_info['id_titulo'] = title_id
            temp_data_list.append(title_info)

        df = pd.json_normalize(
            data=temp_data_list,
            record_path='prospects',
            meta=['id_titulo', 'titulo', 'modalidade'],
            sep='_'
        )
        df.rename(columns={'codigo': 'id_candidato_prospect'}, inplace=True)

        # Converte o ID do título e do candidato prospect para string para merge
        df['id_titulo'] = df['id_titulo'].astype(str)
        df['id_candidato_prospect'] = df['id_candidato_prospect'].astype(str)

        return df
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Erro ao carregar prospects: {e}")
        return pd.DataFrame()

# 3 Combinando Dataframes para análise das features


In [29]:
# --- Caminhos dos arquivos (CORRIGIDO) ---
applicants_file = 'bases/applicants.json' # CORRIGIDO: Nome do arquivo de candidatos
vagas_file = 'bases/vagas.json'
prospects_file = 'bases/prospects.json'

# --- Carregar os DataFrames ---
print("Carregando DataFrames...")
df_applicants = load_json_to_df_applicants(applicants_file) # Usando a função e variável corretas
df_vagas = load_json_to_df_vagas(vagas_file)
df_prospects = load_json_to_df_prospects(prospects_file)

if df_applicants.empty or df_vagas.empty or df_prospects.empty:
    print("Um ou mais DataFrames não puderam ser carregados. Verifique os erros acima.")
    exit()

print(f"df_applicants carregado. Linhas: {len(df_applicants)}")
print(f"df_vagas carregado. Linhas: {len(df_vagas)}")
print(f"df_prospects carregado. Linhas: {len(df_prospects)}")

# --- 1. Combinar df_prospects com df_applicants ---
# A coluna 'id_candidato_prospect' em df_prospects corresponde ao índice 'id_candidato' em df_applicants.
print("\nCombinando prospects com applicants...")
df_applicants_reset = df_applicants.reset_index() # Reseta o índice para usar 'id_candidato' como coluna

df_combined = pd.merge(
    df_prospects,
    df_applicants_reset,
    left_on='id_candidato_prospect',
    right_on='id_candidato',
    how='left',
    suffixes=('_prospect', '_applicant') # Sufixos ajustados para 'applicant'
)

# Remover a coluna id_candidato duplicada após o merge, se necessário
if 'id_candidato_applicant' in df_combined.columns:
    df_combined.drop(columns=['id_candidato_applicant'], inplace=True)

print(f"Combinado prospects e applicants. Linhas: {len(df_combined)}")

# --- 2. Combinar o resultado com df_vagas ---
# A coluna 'id_titulo' em df_combined corresponde ao índice 'id_vaga' em df_vagas.
print("Combinando com vagas...")
df_vagas_reset = df_vagas.reset_index()

df_final = pd.merge(
    df_combined,
    df_vagas_reset,
    left_on='id_titulo',
    right_on='id_vaga',
    how='left',
    suffixes=('_combined', '_vaga') # Sufixos para a segunda combinação
)

# Remover a coluna id_vaga duplicada após o merge, se necessário
if 'id_vaga_vaga' in df_final.columns:
    df_final.drop(columns=['id_vaga_vaga'], inplace=True)

print(f"Combinação final concluída. Linhas: {len(df_final)}")

# --- Exibir o DataFrame Combinado ---
print("\n--- DataFrame Combinado (Primeiras 5 linhas) ---")
print(df_final.head())

print("\n--- Colunas do DataFrame Combinado ---")
print(sorted(df_final.columns.tolist()))

print("\nCombinação de DataFrames concluída! Estamos prontos para a Feature Engineering.")

Carregando DataFrames...
df_applicants carregado. Linhas: 42482
df_vagas carregado. Linhas: 14081
df_prospects carregado. Linhas: 53759

Combinando prospects com applicants...
Combinado prospects e applicants. Linhas: 53759
Combinando com vagas...
Combinação final concluída. Linhas: 53759

--- DataFrame Combinado (Primeiras 5 linhas) ---
              nome_prospect id_candidato_prospect  \
0               José Vieira                 25632   
1  Srta. Isabela Cavalcante                 25529   
2     Sra. Yasmin Fernandes                 25364   
3            Alexia Barbosa                 25360   
4            Arthur Almeida                 26338   

            situacao_candidado data_candidatura ultima_atualizacao  \
0  Encaminhado ao Requisitante       25-03-2021         25-03-2021   
1  Encaminhado ao Requisitante       22-03-2021         23-03-2021   
2     Contratado pela Decision       17-03-2021         12-04-2021   
3  Encaminhado ao Requisitante       17-03-2021         17-03